<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Auth" data-toc-modified-id="Auth-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Auth</a></span></li><li><span><a href="#Create" data-toc-modified-id="Create-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create</a></span></li><li><span><a href="#Update" data-toc-modified-id="Update-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Update</a></span></li><li><span><a href="#Read" data-toc-modified-id="Read-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Read</a></span></li><li><span><a href="#Delete" data-toc-modified-id="Delete-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Delete</a></span></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['SPARK_HOME'] = '/opt/spark/versions/spark-3.1'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/opt/share/secrets/google-application-credential.json"

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import json
from os.path import join as PJ
from typing import List

In [6]:
import pygsheets

# Auth

In [14]:
# auth stuff

gc = pygsheets.authorize(service_account_file='/opt/share/secrets/google-application-credential.json')

In [30]:
print(dir(gc))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'create', 'drive', 'get_range', 'logger', 'oauth', 'open', 'open_all', 'open_as_json', 'open_by_key', 'open_by_url', 'run_batch', 'set_batch_mode', 'sheet', 'spreadsheetId', 'spreadsheet_cls', 'spreadsheet_ids', 'spreadsheet_titles', 'teamDriveId']


In [66]:
# gc.drive.enable_team_drive()

# Create

Create in drive should be easier

In [29]:
# It can do that! with some effert.

joe_sh = gc.create(title='Joe_Test')

#  A permission role can only be one of ['organizer', 'owner', 'writer', 'commenter', 'reader'].
# also you can share a domain
joe_sh.share('joetsai@pixnet.tw',role='writer')

print('url of the sheets : ', joe_sh.url)
print('title of the sheets : ', joe_sh.title)

print('class and method : ')

print(joe_sh, dir(joe_sh), sep='\n\n')

# the sheets create by the service account, so you might have no right to check the sheets

url of the sheets :  https://docs.google.com/spreadsheets/d/1j_5dzuzF4CCv8rI1ABAbU-C5jIbDbU0nOaqjeZ7cMos
title of the sheets :  Joe_Test
class and method : 
<Spreadsheet 'Joe_Test' Sheets:1>

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_defaultFormat', '_fetch_sheets', '_id', '_jsonsheet', '_named_ranges', '_sheet_list', '_title', 'add_worksheet', 'client', 'create_developer_metadata', 'custom_request', 'default_parse', 'defaultformat', 'del_worksheet', 'delete', 'export', 'fetch_properties', 'find', 'get_developer_metadata', 'id', 'logger', 'named_ranges', 'permissions', 'protected_ranges', 'remove_permission', 'replace', 'share', 'sheet1', 'title', 'to_json', '

# Update

In [19]:
# Open spreadsheet and then worksheet

# return a sheets
# need to setup the crenditial
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1KLzEsuFdx2_9o1Kf7GgR_S-iBQQ20ZDHWpMClKDVtWE/edit#gid=0')

print(type(sh), sh)

# working sheets

wks = sh.sheet1

# Update a cell with value (just to let him know values is updated ;) )
wks.update_value('A1', "Hey yank this numpy array")
my_nparray = np.random.randint(10, size=(3, 4))

# update the sheet with array
wks.update_values('A2', my_nparray.tolist())

# already update the value =)

# https://docs.google.com/spreadsheets/d/1KLzEsuFdx2_9o1Kf7GgR_S-iBQQ20ZDHWpMClKDVtWE/edit#gid=0

<class 'pygsheets.spreadsheet.Spreadsheet'> <Spreadsheet '未命名的試算表' Sheets:1>


In [57]:
df = (
    pd.DataFrame(
    {
        'note':['','',''],
        'label':['','',''],
        'some_id':['0001','0002','0003'],
        'aonther_id':['1','2','3'],
        'poi_name' : ['abc','defg','sss'],
        'views' : [100, 50, np.nan],
        'click' : [3, 1, 0]
    }
    )
)

try:
    wks = sh.worksheet_by_title('poi_views_v20210804')
except:
    wks = sh.add_worksheet('poi_views_v20210804')
print('Now we have : ',sh.worksheets(), wks)

# write dataframe into sheets
wks.set_dataframe(df, start=(0,0), nan='NotaNumber')

Now we have :  [<Worksheet '工作表1' index:0>, <Worksheet 'poi_views_v20210804' index:1>] <Worksheet 'poi_views_v20210804' index:1>


# Read

In [59]:
# you can also do export(csv) and read
# wks.export(xxx.csv), pd_read_csv(xxx.csv)

labelled_df = wks.get_as_df(
    numerize=True # watch out the 0001 will be conver to 1
)

labelled_df.info()
labelled_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   note        3 non-null      object
 1   label       3 non-null      object
 2   some_id     3 non-null      int64 
 3   aonther_id  3 non-null      int64 
 4   poi_name    3 non-null      object
 5   views       3 non-null      object
 6   click       3 non-null      int64 
dtypes: int64(3), object(4)
memory usage: 296.0+ bytes


,note,label,some_id,aonther_id,poi_name,views,click
0,,,1,1,abc,100,3
1,,,2,2,defg,50,1
2,無法判斷 views,,3,3,sss,NotaNumber,0


# Delete

In [63]:
# sh.del_worksheet(wks)